In [1]:
# pip install datasets

In [5]:
import pandas as pd
import numpy as np
import torch
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, IterableDataset
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from tqdm.auto import tqdm

In [3]:
# Load dataset in streaming mode
ds = load_dataset("MITLL/LADI-v2-dataset", streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.49k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [4]:
print(ds)

IterableDatasetDict({
    train: IterableDataset({
        features: ['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'],
        n_shards: 40
    })
    validation: IterableDataset({
        features: ['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'],
        n_shards: 40
    })
    test: IterableDataset({
        features: ['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'],
        n_shards: 40
    })
})


In [ ]:
first_example = next(iter(ds['train']))
print(first_example.keys())

dict_keys(['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'])


In [ ]:
if 'Labels' in first_example:
  print('Label Exist!')
  print(f" First example label: {first_example['Labels']}")
else:
  print('Label does not exist in this dataset')

Label does not exist in this dataset


In [ ]:
first_example

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1200>,
 'bridges_any': False,
 'buildings_any': False,
 'buildings_affected_or_greater': False,
 'buildings_minor_or_greater': False,
 'debris_any': False,
 'flooding_any': False,
 'flooding_structures': False,
 'roads_any': False,
 'roads_damage': False,
 'trees_any': True,
 'trees_damage': True,
 'water_any': True}

In [ ]:
# List out possible labels
possible_labels = [key for key, value in first_example.items() if key != 'image']

print(f"Possible labels: {possible_labels}")

Possible labels: ['bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any']


In [ ]:
# Iterate through the dataset
for i, example in enumerate(ds['train']):  # Replace 'train' with the correct split
    image = example['image']

    # Check if 'label' exists or access another field
    label = example.get('label', None)  # Safely get 'label', or None if it doesn't exist

    # If no direct 'label', check other features
    if label is None:
        # Multi-label scenario: list of possible binary labels
        label = {key: value for key, value in example.items() if key != 'image'}

    # Print image and label
    print(f"Image {i}: {image}")
    print(f"Label {i}: {label}")

    if i >= 5:  # Stop after 5 examples
        break

Image 0: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1200 at 0x7FA2C8474A30>
Label 0: {'bridges_any': False, 'buildings_any': False, 'buildings_affected_or_greater': False, 'buildings_minor_or_greater': False, 'debris_any': False, 'flooding_any': False, 'flooding_structures': False, 'roads_any': False, 'roads_damage': False, 'trees_any': True, 'trees_damage': True, 'water_any': True}
Image 1: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1200 at 0x7FA2C8475180>
Label 1: {'bridges_any': False, 'buildings_any': True, 'buildings_affected_or_greater': False, 'buildings_minor_or_greater': False, 'debris_any': False, 'flooding_any': False, 'flooding_structures': False, 'roads_any': True, 'roads_damage': False, 'trees_any': True, 'trees_damage': False, 'water_any': False}
Image 2: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1196 at 0x7FA2C8475780>
Label 2: {'bridges_any': False, 'buildings_any': False, 'buildings_affected_or_greater': False, 'buildings_

In [6]:
# Load the processor and model
processor = AutoImageProcessor.from_pretrained("MITLL/LADI-v2-classifier-large")
model = AutoModelForImageClassification.from_pretrained("MITLL/LADI-v2-classifier-large")

preprocessor_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/781M [00:00<?, ?B/s]

In [7]:
# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=6, bias=False)
                  )
                  (query): Linear(in_features=192, out_features=192, bias=True)
                  (key): Linear(in_features=192, out_featur

In [8]:
# Define optimizer and scaler for mixed precision
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scaler = GradScaler()  # Scaler for mixed precision

<ipython-input-8-63867a461fc3>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Scaler for mixed precision


In [9]:
# Define your label keys
label_keys = ['bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater',
              'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage',
              'trees_any', 'trees_damage', 'water_any']

In [10]:
# Define batch size
batch_size = 3

In [12]:
# Functions to handle streaming data and preprocess images into tensors
class StreamDataset(IterableDataset):
    def __init__(self, dataset, split_name, label_keys, processor):
        self.dataset = dataset
        self.split_name = split_name
        self.label_keys = label_keys
        self.processor = processor

    def process_item(self, item):
        image = item['image']
        labels = [int(item[key]) for key in self.label_keys]

        # Preprocess the image into a tensor using the processor (resize, normalize, etc.)
        processed_image = self.processor(image, return_tensors="pt")['pixel_values'].squeeze(0)  # Remove extra batch dimension
        return processed_image, labels

    def __iter__(self):
        for item in self.dataset[self.split_name]:
            yield self.process_item(item)

# Function to process the dataset
def process_dataset(model, dataset, split_name, label_keys, processor, optimizer=None, train=False, batch_size=8):
    # Set the model's mode
    model.train() if train else model.eval()

    running_loss = 0.0
    all_labels = []
    all_preds = []

    # Use DataLoader to handle batching efficiently
    processed_dataset = StreamDataset(dataset, split_name, label_keys, processor)
    loader = DataLoader(processed_dataset, batch_size=batch_size, collate_fn=lambda x: tuple(zip(*x)))

    # For validation, turn off gradients
    if not train:
        torch.no_grad()

    for batch_images, batch_labels in tqdm(loader):
        batch_images = torch.stack(batch_images).to(device)
        batch_labels = torch.tensor(batch_labels, dtype=torch.float32).to(device)

        # Only compute gradients for training, not validation
        if train:
            with autocast():  # Enable mixed precision
                outputs = model(batch_images, labels=batch_labels)
                loss = outputs.loss

            running_loss += loss.item()

            # Backward pass and optimization for training
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            with torch.no_grad():  # No gradients in validation
                outputs = model(batch_images, labels=batch_labels)
                loss = outputs.loss
                running_loss += loss.item()

        # Get predictions (sigmoid to convert logits to probabilities)
        logits = outputs.logits
        predictions = torch.sigmoid(logits).cpu().detach().numpy()
        all_preds.extend(predictions)
        all_labels.extend(batch_labels.cpu().numpy())

        # Clear GPU cache to avoid memory leaks
        torch.cuda.empty_cache()

    return running_loss / len(all_preds), all_labels, all_preds



In [13]:

# Function to compute evaluation metrics
def compute_metrics(labels, predictions, threshold=0.5):
    # Convert predictions list to NumPy array
    predictions = np.array(predictions)

    # Convert probabilities to binary predictions using threshold
    predictions_bin = (predictions >= threshold).astype(int)

    # Calculate accuracy, precision, recall, f1, and ROC AUC
    accuracy = accuracy_score(labels, predictions_bin)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions_bin, average='macro', zero_division=1)
    roc_auc = roc_auc_score(labels, predictions, average='macro', multi_class='ovr')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

    return accuracy, precision, recall, f1, roc_auc

In [ ]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Training
    train_loss, train_labels, train_preds = process_dataset(model, ds, 'train', label_keys, processor, optimizer, train=True, batch_size=batch_size)
    print(f"Training Loss: {train_loss:.4f}")

    # Free memory after training
    del train_labels, train_preds
    torch.cuda.empty_cache()  # Ensure GPU memory is cleared

    # Validation
    val_loss, val_labels, val_preds = process_dataset(model, ds, 'validation', label_keys, processor, batch_size=batch_size)
    print(f"Validation Loss: {val_loss:.4f}")

    # Compute metrics for validation
    compute_metrics(val_labels, val_preds)

    # Free memory after validation
    del val_labels, val_preds
    torch.cuda.empty_cache()

Epoch 1/3


0it [00:00, ?it/s]

<ipython-input-12-aca201c95e9a>:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision


Training Loss: 0.0310


0it [00:00, ?it/s]

Validation Loss: 0.0637
Accuracy: 0.5157
Precision: 0.8186
Recall: 0.7598
F1 Score: 0.7837
ROC AUC: 0.9545
Epoch 2/3


0it [00:00, ?it/s]

<ipython-input-12-aca201c95e9a>:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision


In [ ]:
# Test the model
print("Testing the model...")
test_loss, test_labels, test_preds = process_dataset(model, ds, 'test', label_keys, processor, batch_size=batch_size)
print(f"Test Loss: {test_loss:.4f}")

# Compute metrics for the test set
compute_metrics(test_labels, test_preds)

# Free memory after test
del test_labels, test_preds
torch.cuda.empty_cache()
